# Step 2: Private Prediction using Syft Keras - Serving (Server)

In [1]:
from collections import OrderedDict

import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import AveragePooling2D, Conv2D, Dense, Activation, Flatten, ReLU, Activation
import tf_encrypted as tfe

import syft as sy
hook = sy.KerasHook(tf.keras)

# alice = TensorFlowWorker(hostmap='localhost:4000')
# bob = TensorFlowWorker(hostmap='localhost:4001')
# carol = TensorFlowWorker(hostmap='localhost:4002')

## Protocol

In [2]:
players = OrderedDict([
    ('server0', 'localhost:4000'),
    ('server1', 'localhost:4001'),
    ('server2', 'localhost:4002'),
])

config = tfe.RemoteConfig(players)
config.save('/tmp/tfe.config')

In [3]:
tfe.set_config(config)

## Launching servers

Before actually serving the computation below we need to launch TensorFlow servers in new processes. Run the following in three different terminals. You may have to allow Python to accept incoming connections.

In [4]:
import subprocess
subprocess_calls = {}
for player_name in players.keys():
    subprocess_calls[player_name] = subprocess.Popen("python -m tf_encrypted.player --config /tmp/tfe.config {}".format(player_name).split(' '))

## Model

In [5]:
task_classes = 10
task_shape = [1, 28, 28, 1]
pre_trained_weights = 'short-conv-mnist.h5'

In [6]:
model = Sequential()

model.add(Conv2D(10, (3, 3), batch_input_shape=task_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(task_classes, name="logit"))

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


### Load pre-trained weights

In [7]:
model.load_weights(pre_trained_weights)

### Secure the model by sharing the weights

In [8]:
prot = tfe.protocol.SecureNN()
model.share(prot)

INFO:tf_encrypted:Starting session on target 'grpc://localhost:4000' using config graph_options {
}



### Serve model

In [9]:
model.serve(num_steps=3)

Served
Served
Served
